# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 62 new papers today
          26 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/25 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.06476


extracting tarball to tmp_2210.06476...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.06482


extracting tarball to tmp_2210.06482...

...

extracting tarball to tmp_2210.07162...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.07163


extracting tarball to tmp_2210.07163...

 done.


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.07163/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Sec999_appendix' from 'tmp_2210.07163/Sec999_appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Sec7_conclusion' from 'tmp_2210.07163/Sec7_conclusion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/si

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06504-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06504) | **Lyman-alpha Emitters in Ionized Bubbles: Constraining the Environment  and Ionized Fraction**  |
|| A.C. Trapp, Steven R. Furlanetto, <mark>Frederick B. Davies</mark> |
|*Appeared on*| *2022-10-14*|
|*Comments*| *14 pages, 14 figures, submitted to MNRAS*|
|**Abstract**| Lyman-alpha emitters (LAEs) are excellent probes of the reionization process, as they must be surrounded by large ionized bubbles in order to be visible during the reionization era. Large ionized regions are thought to correspond to over-dense regions and may be protoclusters, making them interesting test-beds for early massive structures. Close associations containing several LAEs are often assumed to mark over-dense, ionized bubbles. Here, we develop the first framework to quantify the ionization and density fields of high-z galaxy associations. We explore the interplay between (i) the large-scale density of a survey field, (ii) Poisson noise due to the small number density of bright sources at high redshifts (z~7), and (iii) the effects of the ionized fraction on the observation of LAEs. We use Bayesian statistics, a simple model of reionization, and a Monte-Carlo simulation to construct a more comprehensive method for calculating the large-scale density of LAE regions than previous works. We find that Poisson noise has a strong effect on the inferred density of a region and show how the ionized fraction can be inferred. We then apply our framework to the strongest association yet identified: Hu et al. (2021) found 14 LAEs in a volume of ~50,000 cMpc^3 inside the COSMOS field at z~7. We show that this is most likely a 2.5-sigma over-density inside of an ionized or nearly ionized bubble. We also show that this LAE association implies that the global ionized fraction is Q = 0.60 (+0.08,-0.09), within the context of a simple reionization model. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06492-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06492) | **The ASAS-SN Bright Supernova Catalog -- V. 2018-2020**  |
|| <mark>K. D. Neumann</mark>, et al. -- incl., <mark>J. S. Brown</mark> |
|*Appeared on*| *2022-10-14*|
|*Comments*| *14 pages, 7 figures, 4 tables. MNRAS submission in progress. Tables containing the catalog data presented in this submission are included in machine-readable format as ancillary files*|
|**Abstract**| We catalog the 443 bright supernovae discovered by the All-Sky Automated Survey for Supernovae (ASAS-SN) in $2018-2020$ along with the 519 supernovae recovered by ASAS-SN and 516 additional $m_{peak}\leq18$ mag supernovae missed by ASAS-SN. Our statistical analysis focuses primarily on the 984 supernovae discovered or recovered in ASAS-SN $g$-band observations. The complete sample of 2427 ASAS-SN supernovae includes earlier $V$-band samples and unrecovered supernovae. For each supernova, we identify the host galaxy, its UV to mid-IR photometry, and the offset of the supernova from the center of the host. Updated light curves, redshifts, classifications, and host galaxy identifications supersede earlier results. With the increase of the limiting magnitude to $g\leq18$ mag, the ASAS-SN sample is roughly complete up to $m_{peak}=16.7$ mag and is $90\%$ complete for $m_{peak}\leq17.0$ mag. This is an increase from the $V$-band sample where it was roughly complete up to $m_{peak}=16.2$ mag and $70\%$ complete for $m_{peak}\leq17.0$ mag. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06731-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06731) | **Near-infrared Chemical Abundances of Stars in the Sculptor Dwarf Galaxy**  |
|| Baitian Tang, et al. -- incl., <mark>Jiajun Zhang</mark>, <mark>Zhiyu Zhang</mark> |
|*Appeared on*| *2022-10-14*|
|*Comments*| *Resubmitted to A&A after addressing the second referee report*|
|**Abstract**| Owing to the recent identification of major substructures in our Milky Way (MW), the astronomical community start to reevaluate the importance of dissolved and existing dwarf galaxies. In this work, we investigate up to 13 elements in 43 giant stars of Sculptor dwarf galaxy (Scl) using high signal-to-noise ratio near-infrared APOGEE spectra. Thanks to the strong feature lines in the near-infrared, high resolution O, Si, and Al abundances are determined at such large group of sample stars for the first time in Scl. By comparing [$\alpha$/Fe] (i.e., O, Mg, Si, Ca, and Ti) of stars from Scl, Sagittarius, and MW, we confirm the general trend that less massive galaxy tends to show lower [$\alpha$/Fe]. The low [Al/Fe] ($\sim -0.5$) in Scl proves it as a trustworthy discriminator to identify stars born in dwarf galaxies (from MW field stars). Chemical evolution model suggests Scl have a top-light initial mass function (IMF), with a high-mass IMF power index $\sim -2.7$, and a minimum SN Ia delay time of $\sim 100$ Myr. Furthermore, linear regression analysis indicates negative radial metallicity gradient and positive radial gradients for [Mg/Fe] and [Ca/Fe], qualitatively agree with the outside-in formation scenario. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07080-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07080) | **Measuring Noise Parameters Using an Open, Short, Load, and  1/8-wavelength Cable as Source Impedances**  |
|| Danny C. Price, et al. -- incl., <mark>Lincoln J. Greenhill</mark> |
|*Appeared on*| *2022-10-14*|
|*Comments*| *IEEE Transactions on Microwave Theory and Techniques*|
|**Abstract**| Noise parameters are a set of four measurable quantities which determine the noise performance of a radio-frequency device under test. The noise parameters of a 2-port device can be extracted by connecting a set of 4 or more source impedances at the device's input, measuring the noise power of the device with each source connected, and then solving a matrix equation. However, sources with high reflection coefficients cannot be used due to a singularity that arises in entries of the matrix. Here, we detail a new method of noise parameter extraction using a singularity-free matrix that is compatible with high-reflection sources. We show that open, short, load and an open cable ("OSLC") can be used to extract noise parameters, and we detail a practical measurement approach. The OSLC approach is particularly well-suited for low-noise amplifier measurement at frequencies below 1 GHz, where alternative methods require physically large apparatus. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07149-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07149) | **Searching for compact objects in the single-lined spectroscopic binaries  of the young Galactic cluster NGC 6231**  |
|| G. Banyard, et al. -- incl., <mark>L. Mahy</mark>, <mark>N. Langer</mark> |
|*Appeared on*| *2022-10-14*|
|*Comments*| *Submitted to A&A*|
|**Abstract**| Recent evolutionary computations predict that a few percent of massive OB stars in binary systems should have a dormant BH companion. Despite several reported X-ray quiet OB+BH systems over the last couple of years, finding them with certainty remains challenging. These have great importance as they can be gravitational wave (GW) source progenitors, and are landmark systems in constraining supernova kick physics. This work aims to characterise the hidden companions to the single-lined spectroscopic binaries (SB1s) in the B star population of the young open Galactic cluster NGC 6231 to find candidate systems for harbouring compact object companions. With the orbital solutions for each SB1 previously constrained, we applied Fourier spectral disentangling to multi-epoch optical VLT/FLAMES spectra of each target to extract a potential signature of a faint companion, and to identify newly disentangled double-lined spectroscopic binaries (SB2s). For targets where the disentangling does not reveal any signature of a stellar companion, we performed atmospheric and evolutionary modelling on the primary to obtain constraints on the unseen companion. Seven newly classified SB2 systems with mass ratios down to near 0.1 were identified. From the remaining targets, for which no faint companion could be extracted from the spectra, four are found to have companion masses in the predicted mass ranges of neutron stars (NSes) and BHes. Two of these have companion masses between 1 and 3.5 $M_{\odot}$, making them potential hosts of NSes (or lower mass main sequence stars). The other two are between 2.5 to 8 $M_{\odot}$ and 1.6 and 26 $M_{\odot}$, respectively, and so are identified as candidates for harbouring BH companions. However, unambiguous identification of these systems as X-ray quiet compact object harbouring binaries requires follow up observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07162-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07162) | **Probing the interaction between dark energy and dark matter with future  fast radio burst observations**  |
|| <mark>Ze-Wei Zhao</mark>, <mark>Ling-Feng Wang</mark>, <mark>Ji-Guo Zhang</mark>, <mark>Jing-Fei Zhang</mark>, <mark>Xin Zhang</mark> |
|*Appeared on*| *2022-10-14*|
|*Comments*| *16 pages, 6 figures*|
|**Abstract**| Interacting dark energy (IDE) scenario assumes that there exists a direct interaction between dark energy and cold dark matter, but this interaction is hard to be tightly constrained by the current data. Finding new cosmological probes to precisely measure this interaction could deepen our understanding of dark energy and dark matter. Fast radio bursts (FRBs) will be seen in large numbers by future radio telescopes, and thus they have potential to become a promising low-redshift cosmological probe. In this work, we investigate the capability of future FRBs of constraining the dimensionless coupling parameter $\beta$ in four phenomenological IDE models. We find that in the IDE models with the interaction proportional to the energy density of dark energy, about $10^5$ FRB data can give constraint on $\beta$ tighter than the current cosmic microwave background data. In all the IDE models, about $10^6$ FRB data can constrain the absolute errors of $\beta$ to less than $0.10$, providing a way to precisely measure $\beta$ by only one cosmological probe. The reconstruction of the interaction term also shows that the FRB data could help constrain the redshift evolution of interaction. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.07163-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.07163) | **KiDS-Legacy calibration: unifying shear and redshift calibration with  the SKiLLS multi-band image simulations**  |
|| <mark>Shun-Sheng Li</mark>, et al. -- incl., <mark>Henk Hoekstra</mark>, <mark>Catherine Heymans</mark>, <mark>Hendrik Hildebrandt</mark> |
|*Appeared on*| *2022-10-14*|
|*Comments*| *28 pages, 31 figures, 2 tables, submitted to A&A*|
|**Abstract**| We present SKiLLS, a suite of multi-band image simulations for the KiDS-Legacy analysis, the weak lensing analysis of the complete Kilo-Degree Survey. The resulting catalogues enable joint shear and redshift calibration, enhancing the realism and hence accuracy over previous efforts. To create a large volume of simulated galaxies with faithful properties and to a sufficient depth, we integrate cosmological simulations with high-quality imaging observations. We also improve the realism of simulated images by allowing the point spread function (PSF) to differ between CCD images, including stellar density variations and varying noise levels between pointings. Using realistic variable shear fields, we account for the impact of blended systems at different redshifts. Although the overall correction is minor, we find a clear redshift-bias correlation in the blending-only variable shear simulations, indicating the non-trivial impact of this higher-order blending effect. We also explore the impact of the PSF modelling errors and find a small but noticeable effect on the shear bias. Finally, we conduct a series of sensitivity tests, including changing the input galaxy properties. We conclude that our fiducial shape measurement algorithm, lensfit, is robust within the requirements of lensing analyses with KiDS. As for future weak lensing surveys with tighter requirements, we suggest further investments in understanding the impact of blends at different redshifts, improving the PSF modelling algorithm and developing the shape measurement method to be less sensitive to the galaxy properties. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06563-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06563) | **MAROON-X: The first two years of EPRVs from Gemini North**  |
|| Andreas Seifahrt, et al. -- incl., <mark>Robert Liu</mark> |
|*Appeared on*| *2022-10-14*|
|*Comments*| **|
|**Abstract**| MAROON-X is a fiber-fed, optical EPRV spectrograph at the 8-m Gemini North Telescope on Mauna Kea, Hawai'i. MAROON-X was commissioned as a visiting instrument in December 2019 and is in regular use since May 2020. Originally designed for RV observations of M-dwarfs, the instrument is used for a broad range of exoplanet and stellar science cases and has transitioned to be the second-most requested instrument on Gemini North over a number of semesters. We report here on the first two years of operations and radial velocity observations. MAROON-X regularly achieves sub-m/s RV performance on sky with a short-term instrumental noise floor at the 30 cm/s level. We will discuss various technical aspects in achieving this level of precision and how to further improve long-term performance |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.06504.md
    + _build/html/tmp_2210.06504/./laefig_ionized_denpost_linear_COSMOSNavg.png
    + _build/html/tmp_2210.06504/./laefig_ionized_denpost_linear_LAGERNavg.png
    + _build/html/tmp_2210.06504/./laefig_Fsurvive.png
    + _build/html/tmp_2210.06504/./laefig_zetafcoll.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

#  Emitters in Ionized Bubbles: Constraining the Environment and Ionized Fraction

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.06504-b31b1b.svg)](https://arxiv.org/abs/2210.06504)<mark>Appeared on: 2022-10-14</mark> - _14 pages, 14 figures, submitted to MNRAS_

</div>
<div id="authors">

A.C. Trapp, Steven R. Furlanetto, <mark><mark>Frederick B. Davies</mark></mark>

</div>
<div id="abstract">

**Abstract:** Lyman-alpha emitters (LAEs) are excellent probes of the reionization process, as they must be surrounded by large ionized bubbles in order to be visible during the reionization era. Large ionized regions are thought to correspond to over-dense regions and may beprotoclusters, making them interesting test-beds for early massive structures.Close associations containing several LAEs are often assumed to mark over-dense, ionized bubbles.Here, we develop the first framework to quantify the ionization and density fields of high-$z$galaxy associations.We explore the interplay between\textit{(i)}the large-scale density of a survey field,\textit{(ii)}Poisson noise due to the small number density of bright sources at high redshifts ($z \sim$7), and\textit{(iii)}the effects of the ionized fraction on the observation of LAEs.We use Bayesian statistics, a simple model of reionization, and a Monte-Carlo simulation to construct a more comprehensive method for calculating the large-scale density of LAE regions than previous works.We find that Poisson noise has a strong effect on the inferred density of a regionand show how the ionized fraction can be inferred. We then apply our framework to the strongest association yet identified: Hu et al. (2021) found 14 LAEs in a volume of$\sim$50,000 cMpc$^3$inside the COSMOS field at$z \sim7$. We show that this is most likely a 2.5$\sigma$over-densityinside of an ionized or nearly ionized bubble. We also show that this LAE association implies that the global ionized fraction is$\bar{Q} = 0.60^{+0.08}_{-0.09}$, within the context of a simple reionization model.

</div>

<div id="div_fig1">

<img src="tmp_2210.06504/./laefig_ionized_denpost_linear_COSMOSNavg.png" alt="Fig13.1" width="50%"/><img src="tmp_2210.06504/./laefig_ionized_denpost_linear_LAGERNavg.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** 
    The inferred density  for the \citet{Hu2021} volume, assuming no effect from reionization.  \emph{Left:} The \textit{magenta curve} shows the posterior of the linear density, which has 68.27\% credible interval
    $\delta = 0.17 \pm 0.05$, taking the average source density from the COSMOS field alone.
    The vertical dashed \textit{black} line indicates the approximate density required for this region to virialize by $z = $ 0, $\delta_{\textrm{pc}} = 0.27$, which is disfavored by the posterior. The vertical dashed \textit{orange} line shows the density estimate of \citet{Hu2021}.
    The \textit{red and blue} curves show the likelihood and prior, respectively;
    we are in the prior-dominated regime, given the likelihood's large breadth.
    The likelihood and prior are very far apart, indicating that this volume is a rare find; we show in the text that
    the probability of finding at least one such region in a larger survey of 20 similar volumes (the COSMOS field) is 0.11\%. \emph{Right:} Same, but using the $\Nexpavg$ value from all 4 LAGER fields. The inferred density increases from $\delta = 0.17 \rightarrow 0.19$. The probability of finding at least one such region in a larger survey of 118 similar volumes (all 4 LAGER fields) drops to 0.01\%. (*lae_fig:HuPost_COSMOS*)

</div>
<div id="div_fig2">

<img src="tmp_2210.06504/./laefig_Fsurvive.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
    The fraction of LAEs that are visible $\Fsurvavg$ as a function of average ionization fraction $\bar{Q}$ for various bias values.
    Higher bias values result in a larger fraction of all LAEs being visible for a given ionization fraction of the Universe.
    The \textit{red} line shows the total ionizing photon production rate $\zeta \cdot f_{\textrm{coll,avg}}$, for which a value above the line $x=y$ means photons are escaping their host galaxies but being absorbed before contributing to reionization. The most over-dense $R_\alpha$ regions are overproducing photons but can't help their neighbors reionize. This effect is only large when the Universe is mostly reionized. (*lae_fig:Fsurvive*)

</div>
<div id="div_fig3">

<img src="tmp_2210.06504/./laefig_zetafcoll.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
    Properties of our simple reionization model. The
    \textit{red} curve shows the ionizing efficiency required to ionize an independent region as a function of its collapse fraction (see eq. \ref{lae_eq:Qzetafcoll}).
    The \textit{blue} curve shows $p(f_{\textrm{coll}}|R=R_\alpha)$, the (un-normalized) probability distribution of collapse fractions for regions of size $R = R_\alpha$. The \textit{black dashed} curve shows the  average collapse fraction at $z=$ 7. (*lae_fig:zetafcoll*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

63  publications in the last 7 days.
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers